<a href="https://colab.research.google.com/github/VladShajdulin/OTUS/blob/main/home_work_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [24]:
low, high = -10, 10
n_size = 20000

torch.manual_seed(142)
X = (high - low) * torch.rand((n_size, 2), dtype=torch.float64) + low
y = torch.sin(X[:,0] + 2 * X[:,1]) * torch.exp(-(2 * X[:,0] + X[:,1]) ** 2)
y = torch.unsqueeze(y, -1)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=142)
mu, std, delta = y_train.mean(), y_train.std(), 0.1
y_train, y_test = (y_train - mu + delta) / std, (y_test - mu + delta) / std
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=142)

print(f'Training samples: {X_train.size()[0]}')
print(f'Validating samples: {X_val.size()[0]}')
print(f'Training samples: {X_test.size()[0]}')

Training samples: 14000
Validating samples: 3000
Training samples: 3000


In [26]:
class XY_dataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.size()[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [61]:
def train(dataloader, model, loss_fn, optimizer, device):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      # Compute prediction error
      pred = model(X)
      loss = loss_fn(pred, y)

      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 10 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val(X, y, model, loss_fn):
  model.eval()
  with torch.no_grad():
    pred = model(X)
    loss = loss_fn(pred, y).item()
    error = max(abs(y - pred)).item()

  print(f"\n Val loss: {loss:>7f}, max error: {error:>5f} \n")

In [87]:
model_1 = nn.Sequential(
    nn.Linear(2, 20, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(20, 20, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(20, 10, dtype=torch.float64),
    nn.Softsign(),
    nn.Linear(10, 1, dtype=torch.float64)
)

In [88]:
batch_size = 64

train_loader = XY_dataset(X_train, y_train)
train_loader = DataLoader(train_loader, shuffle=True, batch_size=batch_size)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print('Device', device)

Device cpu


In [89]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model_1.parameters())
epochs = 10

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_loader, model_1, loss_fn, optimizer, device)
  val(X_val, y_val, model_1, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.262879  [   64/14000]
loss: 1.133521  [  704/14000]
loss: 1.108327  [ 1344/14000]
loss: 0.907854  [ 1984/14000]
loss: 0.987849  [ 2624/14000]
loss: 0.617280  [ 3264/14000]
loss: 0.581413  [ 3904/14000]
loss: 0.482333  [ 4544/14000]
loss: 0.463959  [ 5184/14000]
loss: 0.220458  [ 5824/14000]
loss: 0.147505  [ 6464/14000]
loss: 0.531659  [ 7104/14000]
loss: 0.110791  [ 7744/14000]
loss: 0.268849  [ 8384/14000]
loss: 0.113059  [ 9024/14000]
loss: 0.352869  [ 9664/14000]
loss: 0.220894  [10304/14000]
loss: 0.219160  [10944/14000]
loss: 0.377700  [11584/14000]
loss: 0.208319  [12224/14000]
loss: 0.143368  [12864/14000]
loss: 0.174426  [13504/14000]

 Val loss: 0.238697, max error: 7.696066 

Epoch 2
-------------------------------
loss: 0.147500  [   64/14000]
loss: 0.413572  [  704/14000]
loss: 0.152566  [ 1344/14000]
loss: 0.251652  [ 1984/14000]
loss: 0.263142  [ 2624/14000]
loss: 0.122740  [ 3264/14000]
loss: 0.215972  [ 3904/14000]
loss: 

In [71]:
y_test * std + mu - delta

tensor([[ 0.0000],
        [ 0.0000],
        [ 0.1077],
        ...,
        [-0.0017],
        [ 0.0000],
        [ 0.0000]], dtype=torch.float64)